In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import time
import urllib3
import json

In [2]:
import os

url = 'https://www.bcn.cl/leychile/navegar/imprimir?idNorma=242302&idVersion=2022-04-29'

In [3]:
## Leyendo archivos

In [4]:
path = 'Textos/'
filelist = os.listdir(path)

In [5]:
filelist.sort()

In [6]:
filelist

['Capitulo 01.txt',
 'Capitulo 02.txt',
 'Capitulo 03.txt',
 'Capitulo 04.txt',
 'Capitulo 05.txt',
 'Capitulo 06.txt',
 'Capitulo 07.txt',
 'Capitulo 08.txt',
 'Capitulo 09.txt',
 'Capitulo 10.txt',
 'Capitulo 11.txt',
 'Capitulo 12.txt',
 'Capitulo 13.txt',
 'Capitulo 14.txt',
 'Capitulo 15.txt',
 'Disposiciones transitorias.txt',
 'Promulgacion.txt']

Eliminando el capitulo de promulgación

In [7]:
filelist.remove('Promulgacion.txt')

In [8]:
filelist[:-1]

['Capitulo 01.txt',
 'Capitulo 02.txt',
 'Capitulo 03.txt',
 'Capitulo 04.txt',
 'Capitulo 05.txt',
 'Capitulo 06.txt',
 'Capitulo 07.txt',
 'Capitulo 08.txt',
 'Capitulo 09.txt',
 'Capitulo 10.txt',
 'Capitulo 11.txt',
 'Capitulo 12.txt',
 'Capitulo 13.txt',
 'Capitulo 14.txt',
 'Capitulo 15.txt']

In [9]:
texto_total = ''

for file in filelist[:-1]:
    with open(path+file, 'r', encoding = 'utf-8') as f:
        text = f.read()
        if text[-1:] != '\n':
            text = text+'\n'
        else:
            text = text
    
    texto_total = texto_total + text

serie = texto_total.split('\n')

In [10]:
df = pd.DataFrame({'texto':serie})
df['texto'] = df['texto'].str.strip()

df['capitulo'] = np.where(df['texto'].str.contains('Preámbulo'),df['texto'],np.nan)
df['capitulo'] = np.where(df['texto'].str.contains('DISPOSICIONES TRANSITORIAS'),df['texto'],df['capitulo'])
df['capitulo'] = np.where(df['texto'].str.contains('Capítulo'),df['texto'],df['capitulo'])
df['capitulo'] = np.where(df['texto'].str.contains('DECIMOSEXTA'),np.nan,df['capitulo'])
df['capitulo'] = np.where(df['texto'].str.contains('Los procesos iniciados, de oficio o a petición de parte, o que se iniciaren en la Corte Suprema'),np.nan,df['capitulo'])
df['capitulo'] = np.where(df['texto'].str.contains('Las contiendas de competencia actualmente trabadas ante la Corte Suprema'),np.nan,df['capitulo'])
df['capitulo'] = np.where(df['texto'].str.contains('En lo no previsto en este Capítulo, serán aplicables'),np.nan,df['capitulo'])
#Para asegurar que aparezca el capitulo IV
df['articulo'] = np.where(df['texto'].str.contains('Artículo'),df['texto'],np.nan)
df['texto'] = np.where(df['texto']=='', np.nan, df['texto'])
df['texto'] = np.where(df['texto']==df['articulo'], np.nan, df['texto'])
df['texto'] = np.where(df['texto']==df['capitulo'], np.nan, df['texto'])
# Agregar numeral y ordinal (??)
df['nro_articulo'] = df['articulo'].str.split('.- ', expand = True)[0]
df['txt_articulo'] = df['articulo'].str.split('.- ', expand = True)[1]

In [11]:
df['capitulo'].fillna(method='pad', inplace=True)
df['nro_articulo'].fillna(method='pad', inplace=True)

In [12]:
df

,texto,capitulo,articulo,nro_articulo,txt_articulo
0,NaN,Capítulo I:BASES DE LA INSTITUCIONALIDAD,NaN,NaN,None
1,NaN,Capítulo I:BASES DE LA INSTITUCIONALIDAD,NaN,NaN,None
2,NaN,Capítulo I:BASES DE LA INSTITUCIONALIDAD,NaN,NaN,None
3,NaN,Capítulo I:BASES DE LA INSTITUCIONALIDAD,NaN,NaN,None
4,NaN,Capítulo I:BASES DE LA INSTITUCIONALIDAD,Artículo 1°.- Las personas nacen libres e igua...,Artículo 1°,Las personas nacen libres e iguales en dignida...
...,...,...,...,...,...
1282,NaN,Capítulo XV:REFORMA DE LA CONSTITUCIÓN Y DEL P...,NaN,Artículo 142,None
1283,NaN,Capítulo XV:REFORMA DE LA CONSTITUCIÓN Y DEL P...,Artículo 143.- Remisión.,Artículo 143,Remisión.
1284,Al plebiscito constitucional le será aplicable...,Capítulo XV:REFORMA DE LA CONSTITUCIÓN Y DEL P...,NaN,Artículo 143,None
1285,Sólo para efecto de ejecutar las acciones en m...,Capítulo XV:REFORMA DE LA CONSTITUCIÓN Y DEL P...,NaN,Artículo 143,None


In [13]:
df['texto'] = np.where(df['texto'].isna(), df['txt_articulo'], df['texto'])

In [14]:
df.dropna(subset = 'texto', inplace = True)

In [15]:
df['nro_capitulo'] = df['capitulo'].str.split(':', expand = True)[0]
df['nro_capitulo'] = df['nro_capitulo'].str.replace('# ','')
df['titulo_capitulo'] = df['capitulo'].str.split(':', expand = True)[1]
df['titulo_capitulo'] = df['titulo_capitulo'].str.strip()

df['titulo_capitulo'] = np.where(df['titulo_capitulo'].isna(), df['nro_capitulo'], df['titulo_capitulo'])

In [16]:
df = df[['nro_capitulo','titulo_capitulo','nro_articulo','texto']]
df.rename(columns = {'nro_capitulo':'capitulo'}, inplace = True)

/tmp/ipykernel_11928/3615833820.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'nro_capitulo':'capitulo'}, inplace = True)


In [17]:
df.groupby(['capitulo','titulo_capitulo']).count()

,,nro_articulo,texto
capitulo,titulo_capitulo,,
Capítulo I,BASES DE LA INSTITUCIONALIDAD,25,25
Capítulo II,NACIONALIDAD Y CIUDADANIA,33,33
Capítulo III,DE LOS DERECHOS Y DEBERES CONSTITUCIONALES,125,125
Capítulo IV,GOBIERNO,96,96
Capítulo IX,SERVICIO ELECTORAL Y JUSTICIA ELECTORAL,18,18
Capítulo V,CONGRESO NACIONAL,179,179
Capítulo VI,PODER JUDICIAL,30,30
Capítulo VII,MINISTERIO PÚBLICO,19,19
Capítulo VIII,TRIBUNAL CONSTITUCIONAL,50,50


In [18]:
df[df['capitulo']=='Capítulo I']

,capitulo,titulo_capitulo,nro_articulo,texto
4,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 1°,Las personas nacen libres e iguales en dignida...
5,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 1°,La familia es el núcleo fundamental de la soci...
6,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 1°,El Estado reconoce y ampara a los grupos inter...
7,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 1°,El Estado está al servicio de la persona human...
8,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 1°,Es deber del Estado resguardar la seguridad na...
11,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 2º,"Son emblemas nacionales la bandera nacional, e..."
14,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 3º,El Estado de Chile es unitario.
15,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 3º,La administración del Estado será funcional y ...
16,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 3º,Los órganos del Estado promoverán el fortaleci...
19,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 4°,Chile es una república democrática.


In [19]:
df[df['capitulo']=='Capítulo II']

,capitulo,titulo_capitulo,nro_articulo,texto
50,Capítulo II,NACIONALIDAD Y CIUDADANIA,Artículo 10,Son chilenos:
51,Capítulo II,NACIONALIDAD Y CIUDADANIA,Artículo 10,"1º.- Los nacidos en el territorio de Chile, co..."
52,Capítulo II,NACIONALIDAD Y CIUDADANIA,Artículo 10,"2º.- Los hijos de padre o madre chilenos, naci..."
53,Capítulo II,NACIONALIDAD Y CIUDADANIA,Artículo 10,3º.- Los extranjeros que obtuvieren carta de n...
54,Capítulo II,NACIONALIDAD Y CIUDADANIA,Artículo 10,4º.- Los que obtuvieren especial gracia de nac...
55,Capítulo II,NACIONALIDAD Y CIUDADANIA,Artículo 10,La ley reglamentará los procedimientos de opci...
58,Capítulo II,NACIONALIDAD Y CIUDADANIA,Artículo 11,La nacionalidad chilena se pierde:
59,Capítulo II,NACIONALIDAD Y CIUDADANIA,Artículo 11,1º.- Por renuncia voluntaria manifestada ante ...
60,Capítulo II,NACIONALIDAD Y CIUDADANIA,Artículo 11,"2º.- Por decreto supremo, en caso de prestació..."
61,Capítulo II,NACIONALIDAD Y CIUDADANIA,Artículo 11,3º.- Por cancelación de la carta de nacionaliz...


In [20]:
df[df['capitulo']=='Capítulo III']

,capitulo,titulo_capitulo,nro_articulo,texto
104,Capítulo III,DE LOS DERECHOS Y DEBERES CONSTITUCIONALES,Artículo 19,La Constitución asegura a todas las personas:
105,Capítulo III,DE LOS DERECHOS Y DEBERES CONSTITUCIONALES,Artículo 19,1º.- El derecho a la vida y a la integridad fí...
106,Capítulo III,DE LOS DERECHOS Y DEBERES CONSTITUCIONALES,Artículo 19,La ley protege la vida del que está por nacer.
107,Capítulo III,DE LOS DERECHOS Y DEBERES CONSTITUCIONALES,Artículo 19,La pena de muerte sólo podrá establecerse por ...
108,Capítulo III,DE LOS DERECHOS Y DEBERES CONSTITUCIONALES,Artículo 19,Se prohíbe la aplicación de todo apremio ilegí...
...,...,...,...,...
228,Capítulo III,DE LOS DERECHOS Y DEBERES CONSTITUCIONALES,Artículo 22,Los chilenos tienen el deber fundamental de ho...
229,Capítulo III,DE LOS DERECHOS Y DEBERES CONSTITUCIONALES,Artículo 22,El servicio militar y demás cargas personales ...
230,Capítulo III,DE LOS DERECHOS Y DEBERES CONSTITUCIONALES,Artículo 22,Los chilenos en estado de cargar armas deberán...
233,Capítulo III,DE LOS DERECHOS Y DEBERES CONSTITUCIONALES,Artículo 23,Los grupos intermedios de la comunidad y sus d...


In [21]:
df[df['capitulo']=='Capítulo IV']

,capitulo,titulo_capitulo,nro_articulo,texto
239,Capítulo IV,GOBIERNO,Artículo 23,Presidente de la República
243,Capítulo IV,GOBIERNO,Artículo 24,El gobierno y la administración del Estado cor...
244,Capítulo IV,GOBIERNO,Artículo 24,Su autoridad se extiende a todo cuanto tiene p...
245,Capítulo IV,GOBIERNO,Artículo 24,"El 1 de junio de cada año, el Presidente de la..."
249,Capítulo IV,GOBIERNO,Artículo 25,Para ser elegido Presidente de la República se...
...,...,...,...,...
402,Capítulo IV,GOBIERNO,Artículo 43,"Por la declaración del estado de emergencia, e..."
406,Capítulo IV,GOBIERNO,Artículo 44,Una ley orgánica constitucional regulará los e...
407,Capítulo IV,GOBIERNO,Artículo 44,Las medidas que se adopten durante los estados...
411,Capítulo IV,GOBIERNO,Artículo 45,Los tribunales de justicia no podrán calificar...


In [22]:
df[df['capitulo']=='Capítulo V']

,capitulo,titulo_capitulo,nro_articulo,texto
417,Capítulo V,CONGRESO NACIONAL,Artículo 46,El Congreso Nacional se compone de dos ramas: ...
421,Capítulo V,CONGRESO NACIONAL,Artículo 46,Composición y generación de la Cámara de Diput...
425,Capítulo V,CONGRESO NACIONAL,Artículo 47,La Cámara de Diputados está integrada por miem...
426,Capítulo V,CONGRESO NACIONAL,Artículo 47,La Cámara de Diputados se renovará en su total...
431,Capítulo V,CONGRESO NACIONAL,Artículo 48,Para ser elegido diputado se requiere ser ciud...
...,...,...,...,...
711,Capítulo V,CONGRESO NACIONAL,Artículo 74,El Presidente de la República podrá hacer pres...
712,Capítulo V,CONGRESO NACIONAL,Artículo 74,La calificación de la urgencia corresponderá h...
716,Capítulo V,CONGRESO NACIONAL,Artículo 75,Si el Presidente de la República no devolviere...
717,Capítulo V,CONGRESO NACIONAL,Artículo 75,La promulgación deberá hacerse siempre dentro ...


In [23]:
df[df['capitulo']=='Capítulo VI']

,capitulo,titulo_capitulo,nro_articulo,texto
723,Capítulo VI,PODER JUDICIAL,Artículo 76,La facultad de conocer de las causas civiles y...
724,Capítulo VI,PODER JUDICIAL,Artículo 76,Reclamada su intervención en forma legal y en ...
725,Capítulo VI,PODER JUDICIAL,Artículo 76,"Para hacer ejecutar sus resoluciones, y practi..."
726,Capítulo VI,PODER JUDICIAL,Artículo 76,La autoridad requerida deberá cumplir sin más ...
729,Capítulo VI,PODER JUDICIAL,Artículo 77,Una ley orgánica constitucional determinará la...
730,Capítulo VI,PODER JUDICIAL,Artículo 77,La ley orgánica constitucional relativa a la o...
731,Capítulo VI,PODER JUDICIAL,Artículo 77,La Corte Suprema deberá pronunciarse dentro de...
732,Capítulo VI,PODER JUDICIAL,Artículo 77,"Sin embargo, si el Presidente de la República ..."
733,Capítulo VI,PODER JUDICIAL,Artículo 77,"En dicho caso, la Corte deberá evacuar la cons..."
734,Capítulo VI,PODER JUDICIAL,Artículo 77,Si la Corte Suprema no emitiere opinión dentro...


In [24]:
df[df['capitulo']=='Capítulo VII']

,capitulo,titulo_capitulo,nro_articulo,texto
773,Capítulo VII,MINISTERIO PÚBLICO,Artículo 83,"Un organismo autónomo, jerarquizado, con el no..."
774,Capítulo VII,MINISTERIO PÚBLICO,Artículo 83,El ofendido por el delito y las demás personas...
775,Capítulo VII,MINISTERIO PÚBLICO,Artículo 83,El Ministerio Público podrá impartir órdenes d...
776,Capítulo VII,MINISTERIO PÚBLICO,Artículo 83,"El ejercicio de la acción penal pública, y la ..."
780,Capítulo VII,MINISTERIO PÚBLICO,Artículo 84,Una ley orgánica constitucional determinará la...
781,Capítulo VII,MINISTERIO PÚBLICO,Artículo 84,La ley orgánica constitucional establecerá el ...
785,Capítulo VII,MINISTERIO PÚBLICO,Artículo 85,El Fiscal Nacional será designado por el Presi...
786,Capítulo VII,MINISTERIO PÚBLICO,Artículo 85,El Fiscal Nacional deberá tener a lo menos die...
787,Capítulo VII,MINISTERIO PÚBLICO,Artículo 85,Será aplicable al Fiscal Nacional lo dispuesto...
791,Capítulo VII,MINISTERIO PÚBLICO,Artículo 86,Existirá un Fiscal Regional en cada una de las...


In [25]:
df[df['capitulo']=='Capítulo VIII']

,capitulo,titulo_capitulo,nro_articulo,texto
820,Capítulo VIII,TRIBUNAL CONSTITUCIONAL,Artículo 92,Habrá un Tribunal Constitucional integrado por...
821,Capítulo VIII,TRIBUNAL CONSTITUCIONAL,Artículo 92,a) Tres designados por el Presidente de la Rep...
822,Capítulo VIII,TRIBUNAL CONSTITUCIONAL,Artículo 92,b) Cuatro elegidos por el Congreso Nacional. D...
823,Capítulo VIII,TRIBUNAL CONSTITUCIONAL,Artículo 92,c) Tres elegidos por la Corte Suprema en una v...
824,Capítulo VIII,TRIBUNAL CONSTITUCIONAL,Artículo 92,Los miembros del Tribunal durarán nueve años e...
825,Capítulo VIII,TRIBUNAL CONSTITUCIONAL,Artículo 92,Los miembros del Tribunal Constitucional serán...
826,Capítulo VIII,TRIBUNAL CONSTITUCIONAL,Artículo 92,En caso que un miembro del Tribunal Constituci...
827,Capítulo VIII,TRIBUNAL CONSTITUCIONAL,Artículo 92,El Tribunal funcionará en pleno o dividido en ...
828,Capítulo VIII,TRIBUNAL CONSTITUCIONAL,Artículo 92,Una ley orgánica constitucional determinará su...
832,Capítulo VIII,TRIBUNAL CONSTITUCIONAL,Artículo 93,Son atribuciones del Tribunal Constitucional:


In [26]:
df[df['capitulo']=='Capítulo IX']

,capitulo,titulo_capitulo,nro_articulo,texto
884,Capítulo IX,SERVICIO ELECTORAL Y JUSTICIA ELECTORAL,Artículo 94 bis,"Un organismo autónomo, con personalidad jurídi..."
885,Capítulo IX,SERVICIO ELECTORAL Y JUSTICIA ELECTORAL,Artículo 94 bis,La dirección superior del Servicio Electoral c...
886,Capítulo IX,SERVICIO ELECTORAL Y JUSTICIA ELECTORAL,Artículo 94 bis,Los Consejeros solo podrán ser removidos por l...
887,Capítulo IX,SERVICIO ELECTORAL Y JUSTICIA ELECTORAL,Artículo 94 bis,La organización y atribuciones del Servicio El...
890,Capítulo IX,SERVICIO ELECTORAL Y JUSTICIA ELECTORAL,Artículo 95,"Un tribunal especial, que se denominará Tribun..."
891,Capítulo IX,SERVICIO ELECTORAL Y JUSTICIA ELECTORAL,Artículo 95,Estará constituido por cinco miembros designad...
892,Capítulo IX,SERVICIO ELECTORAL Y JUSTICIA ELECTORAL,Artículo 95,"a) Cuatro ministros de la Corte Suprema, desig..."
893,Capítulo IX,SERVICIO ELECTORAL Y JUSTICIA ELECTORAL,Artículo 95,b) Un ciudadano que hubiere ejercido el cargo ...
894,Capítulo IX,SERVICIO ELECTORAL Y JUSTICIA ELECTORAL,Artículo 95,Las designaciones a que se refiere la letra b)...
895,Capítulo IX,SERVICIO ELECTORAL Y JUSTICIA ELECTORAL,Artículo 95,Los miembros de este tribunal durarán cuatro a...


In [27]:
df[df['capitulo']=='Capítulo X']

,capitulo,titulo_capitulo,nro_articulo,texto
915,Capítulo X,CONTRALORÍA GENERAL DE LA REPÚBLICA,Artículo 98,Un organismo autónomo con el nombre de Contral...
916,Capítulo X,CONTRALORÍA GENERAL DE LA REPÚBLICA,Artículo 98,El Contralor General de la República deberá te...
921,Capítulo X,CONTRALORÍA GENERAL DE LA REPÚBLICA,Artículo 99,En el ejercicio de la función de control de le...
922,Capítulo X,CONTRALORÍA GENERAL DE LA REPÚBLICA,Artículo 99,"Corresponderá, asimismo, al Contralor General ..."
923,Capítulo X,CONTRALORÍA GENERAL DE LA REPÚBLICA,Artículo 99,Si la representación tuviere lugar con respect...
924,Capítulo X,CONTRALORÍA GENERAL DE LA REPÚBLICA,Artículo 99,"En lo demás, la organización, el funcionamient..."
928,Capítulo X,CONTRALORÍA GENERAL DE LA REPÚBLICA,Artículo 100,Las Tesorerías del Estado no podrán efectuar n...


In [28]:
df[df['capitulo']=='Capítulo XI']

,capitulo,titulo_capitulo,nro_articulo,texto
933,Capítulo XI,"FUERZAS ARMADAS, DE ORDEN Y SEGURIDAD PÚBLICA",Artículo 101,Las Fuerzas Armadas dependientes del Ministeri...
934,Capítulo XI,"FUERZAS ARMADAS, DE ORDEN Y SEGURIDAD PÚBLICA",Artículo 101,Las Fuerzas de Orden y Seguridad Pública están...
935,Capítulo XI,"FUERZAS ARMADAS, DE ORDEN Y SEGURIDAD PÚBLICA",Artículo 101,"Las Fuerzas Armadas y Carabineros, como cuerpo..."
939,Capítulo XI,"FUERZAS ARMADAS, DE ORDEN Y SEGURIDAD PÚBLICA",Artículo 102,La incorporación a las plantas y dotaciones de...
943,Capítulo XI,"FUERZAS ARMADAS, DE ORDEN Y SEGURIDAD PÚBLICA",Artículo 103,"Ninguna persona, grupo u organización podrá po..."
944,Capítulo XI,"FUERZAS ARMADAS, DE ORDEN Y SEGURIDAD PÚBLICA",Artículo 103,Una ley determinará el Ministerio o los órgano...
948,Capítulo XI,"FUERZAS ARMADAS, DE ORDEN Y SEGURIDAD PÚBLICA",Artículo 104,"Los Comandantes en Jefe del Ejército, de la Ar..."
949,Capítulo XI,"FUERZAS ARMADAS, DE ORDEN Y SEGURIDAD PÚBLICA",Artículo 104,"El Presidente de la República, mediante decret..."
953,Capítulo XI,"FUERZAS ARMADAS, DE ORDEN Y SEGURIDAD PÚBLICA",Artículo 105,"Los nombramientos, ascensos y retiros de los o..."
954,Capítulo XI,"FUERZAS ARMADAS, DE ORDEN Y SEGURIDAD PÚBLICA",Artículo 105,"El ingreso, los nombramientos, ascensos y reti..."


In [29]:
df[df['capitulo']=='Capítulo XII']

,capitulo,titulo_capitulo,nro_articulo,texto
959,Capítulo XII,CONSEJO DE SEGURIDAD NACIONAL,Artículo 106,Habrá un Consejo de Seguridad Nacional encarga...
960,Capítulo XII,CONSEJO DE SEGURIDAD NACIONAL,Artículo 106,En los casos que el Presidente de la República...
964,Capítulo XII,CONSEJO DE SEGURIDAD NACIONAL,Artículo 107,El Consejo de Seguridad Nacional se reunirá cu...
965,Capítulo XII,CONSEJO DE SEGURIDAD NACIONAL,Artículo 107,El Consejo no adoptará acuerdos sino para dict...
966,Capítulo XII,CONSEJO DE SEGURIDAD NACIONAL,Artículo 107,"Las actas del Consejo serán públicas, a menos ..."
967,Capítulo XII,CONSEJO DE SEGURIDAD NACIONAL,Artículo 107,Un reglamento dictado por el propio Consejo es...


In [30]:
df[df['capitulo']=='Capítulo XIII']

,capitulo,titulo_capitulo,nro_articulo,texto
972,Capítulo XIII,BANCO CENTRAL,Artículo 108,"Existirá un organismo autónomo, con patrimonio..."
976,Capítulo XIII,BANCO CENTRAL,Artículo 109,El Banco Central sólo podrá efectuar operacion...
977,Capítulo XIII,BANCO CENTRAL,Artículo 109,"Sin perjuicio de lo anterior, en situaciones e..."
978,Capítulo XIII,BANCO CENTRAL,Artículo 109,Ningún gasto público o préstamo podrá financia...
979,Capítulo XIII,BANCO CENTRAL,Artículo 109,"Con todo, en caso de guerra exterior o de peli..."
980,Capítulo XIII,BANCO CENTRAL,Artículo 109,El Banco Central no podrá adoptar ningún acuer...


In [31]:
df[df['capitulo']=='Capítulo XIV']

,capitulo,titulo_capitulo,nro_articulo,texto
983,Capítulo XIV,GOBIERNO Y ADMINISTRACIÓN INTERIOR DEL ESTADO,Artículo 110,Para el gobierno y administración interior del...
984,Capítulo XIV,GOBIERNO Y ADMINISTRACIÓN INTERIOR DEL ESTADO,Artículo 110,"La creación, supresión y denominación de regio..."
988,Capítulo XIV,GOBIERNO Y ADMINISTRACIÓN INTERIOR DEL ESTADO,Artículo 110,Gobierno y Administración Regional
992,Capítulo XIV,GOBIERNO Y ADMINISTRACIÓN INTERIOR DEL ESTADO,Artículo 111,La administración superior de cada región resi...
993,Capítulo XIV,GOBIERNO Y ADMINISTRACIÓN INTERIOR DEL ESTADO,Artículo 111,El gobierno regional estará constituido por un...
...,...,...,...,...
1119,Capítulo XIV,GOBIERNO Y ADMINISTRACIÓN INTERIOR DEL ESTADO,Artículo 126,La ley determinará la forma de resolver las cu...
1120,Capítulo XIV,GOBIERNO Y ADMINISTRACIÓN INTERIOR DEL ESTADO,Artículo 126,"Asimismo, establecerá el modo de dirimir las d..."
1125,Capítulo XIV,GOBIERNO Y ADMINISTRACIÓN INTERIOR DEL ESTADO,Artículo 126,Disposiciones Especiales
1129,Capítulo XIV,GOBIERNO Y ADMINISTRACIÓN INTERIOR DEL ESTADO,Artículo 126 bis,Son territorios especiales los correspondiente...


In [32]:
df[df['capitulo']=='Capítulo XV']

,capitulo,titulo_capitulo,nro_articulo,texto
1135,Capítulo XV,REFORMA DE LA CONSTITUCIÓN Y DEL PROCEDIMIENTO...,Artículo 126 bis,Reforma de la Constitución
1137,Capítulo XV,REFORMA DE LA CONSTITUCIÓN Y DEL PROCEDIMIENTO...,Artículo 127,Los proyectos de reforma de la Constitución po...
1138,Capítulo XV,REFORMA DE LA CONSTITUCIÓN Y DEL PROCEDIMIENTO...,Artículo 127,El proyecto de reforma necesitará para ser apr...
1139,Capítulo XV,REFORMA DE LA CONSTITUCIÓN Y DEL PROCEDIMIENTO...,Artículo 127,"En lo no previsto en este Capítulo, serán apli..."
1143,Capítulo XV,REFORMA DE LA CONSTITUCIÓN Y DEL PROCEDIMIENTO...,Artículo 128,El proyecto que aprueben ambas Cámaras pasará ...
...,...,...,...,...
1280,Capítulo XV,REFORMA DE LA CONSTITUCIÓN Y DEL PROCEDIMIENTO...,Artículo 142,La Constitución deberá imprimirse y repartirse...
1281,Capítulo XV,REFORMA DE LA CONSTITUCIÓN Y DEL PROCEDIMIENTO...,Artículo 142,Si la cuestión planteada al electorado en el p...
1283,Capítulo XV,REFORMA DE LA CONSTITUCIÓN Y DEL PROCEDIMIENTO...,Artículo 143,Remisión.
1284,Capítulo XV,REFORMA DE LA CONSTITUCIÓN Y DEL PROCEDIMIENTO...,Artículo 143,Al plebiscito constitucional le será aplicable...


In [34]:
df['nro_articulo'] = np.where(df['texto']=='Presidente de la República','Artículo 24', df['nro_articulo'])

/tmp/ipykernel_11928/1734335506.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nro_articulo'] = np.where(df['texto']=='Presidente de la República','Artículo 24', df['nro_articulo'])


In [35]:
df_capitulos = df[df['capitulo']!='DISPOSICIONES TRANSITORIAS']

## Disposiciones transitorias

In [36]:
texto_total = ''

for file in filelist[-1:]:
    with open(path+file, 'r', encoding = 'utf-8') as f:
        text = f.read()
        if text[-1:] != '\n':
            text = text+'\n'
        else:
            text = text
    
    texto_total = texto_total + text

serie = texto_total.split('\n')

In [37]:
df = pd.DataFrame({'texto':serie})
df['texto'] = df['texto'].str.strip()

df['titulo_capitulo'] = 'DISPOSICIONES TRANSITORIAS'
df['capitulo'] = 'DISPOSICIONES TRANSITORIAS'
df['nro_capitulo'] = 'DISPOSICIONES TRANSITORIAS'

df['capitulo'] = np.where(df['texto'].str.contains('DECIMOSEXTA'),np.nan,df['capitulo'])
df['capitulo'] = np.where(df['texto'].str.contains('Los procesos iniciados, de oficio o a petición de parte, o que se iniciaren en la Corte Suprema'),np.nan,df['capitulo'])
df['capitulo'] = np.where(df['texto'].str.contains('Las contiendas de competencia actualmente trabadas ante la Corte Suprema'),np.nan,df['capitulo'])
df['capitulo'] = np.where(df['texto'].str.contains('En lo no previsto en este Capítulo, serán aplicables'),np.nan,df['capitulo'])
#Para asegurar que aparezca el capitulo IV
df['articulo'] = np.where(df['texto'].str.contains('Artículo'),df['texto'],np.nan)
df['texto'] = np.where(df['texto']=='', np.nan, df['texto'])
df['texto'] = np.where(df['texto']==df['articulo'], np.nan, df['texto'])
df['texto'] = np.where(df['texto']==df['capitulo'], np.nan, df['texto'])
# Agregar numeral y ordinal (??)
df['nro_articulo'] = df['articulo'].str.split('.- ', expand = True)[0]
df['nro_articulo'] = df['texto'].str.split('.- ', expand = True)[0]
df['texto'] = df['texto'].str.split('.- ', expand = True)[1]

df['nro_articulo'] = np.where(df['nro_articulo']==df['texto'], np.nan, df['nro_articulo'])
df['txt_articulo'] = df['articulo'].str.split('.- ', expand = True)[1]

In [38]:
df['capitulo'].fillna(method='pad', inplace=True)
df['nro_articulo'].fillna(method='pad', inplace=True)

In [40]:
df['texto'] = np.where(df['texto'].isna(), df['txt_articulo'], df['texto'])
df.dropna(subset = 'texto', inplace = True)

df = df[['nro_capitulo','capitulo','titulo_capitulo','nro_articulo','texto']]

df_DT = df[df['capitulo']=='DISPOSICIONES TRANSITORIAS']

In [41]:
df_DT

,nro_capitulo,capitulo,titulo_capitulo,nro_articulo,texto
2,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,PRIMERA,Mientras se dictan las disposiciones que den c...
3,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,SEGUNDA,"Mientras se dicta el nuevo Código de Minería, ..."
7,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,TERCERA,La gran minería del cobre y las empresas consi...
12,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,CUARTA,Se entenderá que las leyes actualmente en vigo...
16,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,QUINTA,No obstante lo dispuesto en el número 6º del a...
20,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,SEXTA,Sin perjuicio de lo dispuesto en el inciso ter...
22,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,SEPTIMA,El indulto particular será siempre procedente ...
26,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,OCTAVA,"Las normas del capítulo VII ""Ministerio Públic..."
32,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,NOVENA,"No obstante lo dispuesto en el artículo 87, en..."
36,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,DECIMA,Las atribuciones otorgadas a las municipalidad...


In [47]:
df_DT = df_DT[['capitulo','titulo_capitulo','nro_articulo','texto']]

## Dataframe final

In [48]:
df_capitulos

,capitulo,titulo_capitulo,nro_articulo,texto
4,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 1°,Las personas nacen libres e iguales en dignida...
5,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 1°,La familia es el núcleo fundamental de la soci...
6,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 1°,El Estado reconoce y ampara a los grupos inter...
7,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 1°,El Estado está al servicio de la persona human...
8,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 1°,Es deber del Estado resguardar la seguridad na...
...,...,...,...,...
1280,Capítulo XV,REFORMA DE LA CONSTITUCIÓN Y DEL PROCEDIMIENTO...,Artículo 142,La Constitución deberá imprimirse y repartirse...
1281,Capítulo XV,REFORMA DE LA CONSTITUCIÓN Y DEL PROCEDIMIENTO...,Artículo 142,Si la cuestión planteada al electorado en el p...
1283,Capítulo XV,REFORMA DE LA CONSTITUCIÓN Y DEL PROCEDIMIENTO...,Artículo 143,Remisión.
1284,Capítulo XV,REFORMA DE LA CONSTITUCIÓN Y DEL PROCEDIMIENTO...,Artículo 143,Al plebiscito constitucional le será aplicable...


In [49]:
df = df_capitulos.append(df_DT)

/tmp/ipykernel_11928/775018754.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df_capitulos.append(df_DT)


In [50]:
df

,capitulo,titulo_capitulo,nro_articulo,texto
4,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 1°,Las personas nacen libres e iguales en dignida...
5,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 1°,La familia es el núcleo fundamental de la soci...
6,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 1°,El Estado reconoce y ampara a los grupos inter...
7,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 1°,El Estado está al servicio de la persona human...
8,Capítulo I,BASES DE LA INSTITUCIONALIDAD,Artículo 1°,Es deber del Estado resguardar la seguridad na...
...,...,...,...,...
369,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,CUADRAGÉSIMA SÉPTIMA,De la participación de las personas con discap...
378,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,CUADRAGÉSIMA OCTAVA,Las declaraciones de candidaturas independient...
382,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,CUADRAGÉSIMA NOVENA,En razón de la postergación de las próximas el...
405,DISPOSICIONES TRANSITORIAS,DISPOSICIONES TRANSITORIAS,QUINCUAGÉSIMA,Sin perjuicio de lo dispuesto en el artículo 6...
